In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
from tqdm import tqdm
import pickle
tqdm.pandas()
import warnings
import catboost as cb
# from sklearn.metrics import mean_squared_error
# from sklearn.metrics import r2_score
from submission_xgboost.utils import *

### Load Initial position and dict of Omni

In [2]:
initial_states = pd.read_csv('C:/Users/isaac/Documents/Challenge_MIT_2/mit_challenge_2/data/phase_1/00000_to_02284-initial_states.csv')
initial_states = pd.concat([initial_states,pd.read_csv('C:/Users/isaac/Documents/Challenge_MIT_2/mit_challenge_2/data/phase_1/02285_to_02357-initial_states.csv')])
initial_states = pd.concat([initial_states,pd.read_csv('C:/Users/isaac/Documents/Challenge_MIT_2/mit_challenge_2/data/phase_1/02358_to_04264-initial_states.csv')])
initial_states = pd.concat([initial_states,pd.read_csv('C:/Users/isaac/Documents/Challenge_MIT_2/mit_challenge_2/data/phase_1/04265_to_05570-initial_states.csv')])
initial_states = pd.concat([initial_states,pd.read_csv('C:/Users/isaac/Documents/Challenge_MIT_2/mit_challenge_2/data/phase_1/05571_to_05614-initial_states.csv')])
initial_states = pd.concat([initial_states,pd.read_csv('C:/Users/isaac/Documents/Challenge_MIT_2/mit_challenge_2/data/phase_1/05615_to_06671-initial_states.csv')])
initial_states['Timestamp'] = pd.to_datetime(initial_states['Timestamp'])

# add last data
df_temp = pd.read_csv('C:/Users/isaac/Documents/Challenge_MIT_2/mit_challenge_2/data/phase_1/06672_to_08118-initial_states.csv')
df_temp['Timestamp'] = pd.to_datetime(df_temp['Timestamp'])
initial_states = pd.concat([initial_states,df_temp])
initial_states = initial_states.set_index('File ID')

In [3]:
initial_states =  initial_states.mask((initial_states==9.99e+29)|(initial_states==9.99e+32))

In [4]:
initial_states = initial_states.loc[initial_states.count(1)==10]

### Estimates lat, long, alt

In [ ]:
model_alt = cb.CatBoostRegressor(loss_function='RMSE',iterations=100,subsample=0.7)
model_alt.fit(X=initial_states.dropna()[['Semi-major Axis (km)',	'Eccentricity',	'Inclination (deg)',	'RAAN (deg)',	'Argument of Perigee (deg)',	'True Anomaly (deg)']],
              y= initial_states.dropna()['Altitude (km)'])

In [ ]:
initial_states['Altitude (km) estimated'] = model_alt.predict(initial_states[['Semi-major Axis (km)',	'Eccentricity',	'Inclination (deg)',	'RAAN (deg)',	'Argument of Perigee (deg)',	'True Anomaly (deg)']])

In [ ]:
initial_states[['Altitude (km)' , 'Altitude (km) estimated']].loc[initial_states['Altitude (km)'].isna()]

In [ ]:
model_lat = cb.CatBoostRegressor(loss_function='RMSE',iterations=100,subsample=0.8,depth=2)
model_lat.fit(X=initial_states.dropna()[['Semi-major Axis (km)',	'Eccentricity',	'Inclination (deg)',	'RAAN (deg)',	'Argument of Perigee (deg)',	'True Anomaly (deg)']],
              y= initial_states.dropna()['Latitude (deg)'])

In [ ]:
model_long = cb.CatBoostRegressor(loss_function='RMSE',iterations=100,depth=2)
model_long.fit(X=initial_states.dropna()[['Semi-major Axis (km)',	'Eccentricity',	'Inclination (deg)',	'RAAN (deg)',	'Argument of Perigee (deg)',	'True Anomaly (deg)']].iloc[:4000],
              y= initial_states.dropna()['Longitude (deg)'].iloc[:4000])

print(model_long.score(X=initial_states.dropna()[['Semi-major Axis (km)',	'Eccentricity',	'Inclination (deg)',	'RAAN (deg)',	'Argument of Perigee (deg)',	'True Anomaly (deg)']].iloc[:4000],
              y= initial_states.dropna()['Longitude (deg)'].iloc[:4000]))

print(model_long.score(X=initial_states.dropna()[['Semi-major Axis (km)',	'Eccentricity',	'Inclination (deg)',	'RAAN (deg)',	'Argument of Perigee (deg)',	'True Anomaly (deg)']].iloc[4000:],
              y= initial_states.dropna()['Longitude (deg)'].iloc[4000:]))

### Read data

In [ ]:
with open('data/data_isaac/omni.pickle', 'rb') as f:
    omni = pickle.load(f)
with open('data/data_isaac/sat.pickle', 'rb') as f:
    sat = pickle.load(f)

for i in sat.keys():
    sat[i]['Orbit Mean Density (kg/m^3)'] = sat[i]['Orbit Mean Density (kg/m^3)'].replace(9.99e32, np.nan)
    sat[i]['Orbit Mean Density (kg/m^3)'] = sat[i]['Orbit Mean Density (kg/m^3)'].ffill().bfill() # add forward and backward fill

In [ ]:
# all_files_omni = os.listdir("phase_1/omni2/")
# all_files_dens = os.listdir("phase_1/sat_density/")
# omni = {}
# for i in tqdm(all_files_omni):
#     omni[int(i.split('-')[1])] = pd.read_csv('phase_1/omni2/'+i)
# sat = {}
# for i in tqdm(all_files_dens):
#     sat[int(i.replace('gr-of','grof').split('-')[1])] = pd.read_csv('phase_1/sat_density/'+i)
# with open('omni.pickle', 'wb') as handle:
#     pickle.dump(omni, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('sat.pickle', 'wb') as handle:
#     pickle.dump(sat, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
res = initial_states.reset_index()['File ID'].progress_apply(lambda win:create_features_and_target(win,initial_states,omni,sat,predict_mean=False,training_mode=True))

In [ ]:
with open('data/data_isaac/res_6000.pickle', 'wb') as handle:
     pickle.dump(res, handle, protocol=pickle.HIGHEST_PROTOCOL)

## XGBOOST

In [5]:
with open('data/data_isaac/res_6000.pickle', 'rb') as f:
    res = pickle.load(f)

### Prediction by mean

In [ ]:
model_mean = cb.CatBoostRegressor(loss_function='RMSE',iterations=10000)
model_mean.fit(df_train.drop(['Timestamp_initial','Target'],axis=1).iloc[:6000],df_train['Target'].iloc[:6000])

In [ ]:
model_mean.score(df_train.drop(['Timestamp_initial','Target'],axis=1).iloc[:6000],df_train['Target'].iloc[:6000])

In [ ]:
model_mean.score(df_train.drop(['Timestamp_initial','Target'],axis=1).iloc[6000:],df_train['Target'].iloc[6000:])

In [ ]:
plt.scatter(model_mean.predict(df_train.drop(['Timestamp_initial','Target'],axis=1).iloc[:6000]),df_train['Target'].iloc[:6000])

In [ ]:
plt.scatter(model_mean.predict(df_train.drop(['Timestamp_initial','Target'],axis=1).iloc[6000:]),df_train['Target'].iloc[6000:])

### Prediction by timestamp to predict

#### Test train validate

In [6]:
index = list(res.keys())
np.random.seed(24)
np.random.shuffle(index)

In [ ]:
#train_index,valid_index,test_index = index[:5600],index[5600:6900],index[6900:]

In [ ]:
'YEAR', 'DOY', 'Hour','Timestamp'

In [7]:
train_index,valid_index,test_index = index[4000:4600],index[4600:5900],index[5900:]
col_to_remove = ['FileId','Timestamp','Timestamp_initial'] +\
                [i for i in res[0].columns if i.find('YEAR')!=-1]+\
                [i for i in res[0].columns if i.find('DOY')!=-1]+\
                [i for i in res[0].columns if i.find('Hour')!=-1]+\
                [i for i in res[0].columns if i.find('std_')!=-1]+\
                [i for i in res[0].columns if i.find('median_')!=-1]
                #[i for i in res[0].columns if i.find('mean_')!=-1]+\
                #[i for i in res[0].columns if i.find('max_')!=-1]+\
                #[i for i in res[0].columns if i.find('min_')!=-1]+\
print(len(col_to_remove))

15


In [ ]:
df_training_set = pd.concat( dict((k,res[k]) for k in train_index if k in initial_states.index)).reset_index().rename({'level_0':'FileId'},axis=1)
df_training_set = df_training_set.drop(col_to_remove,axis=1)
df_training_set = df_training_set.dropna(subset=['Orbit Mean Density (kg/m^3)'],axis=0)

df_valid_set =  pd.concat( dict((k,res[k]) for k in valid_index if k in initial_states.index)).reset_index().rename({'level_0':'FileId'},axis=1)
df_valid_set = df_valid_set.drop(col_to_remove,axis=1)
df_valid_set = df_valid_set.dropna(subset=['Orbit Mean Density (kg/m^3)'],axis=0)

df_test_set =  pd.concat( dict((k,res[k]) for k in test_index if k in initial_states.index)).reset_index().rename({'level_0':'FileId'},axis=1)
df_test_set = df_test_set.drop(col_to_remove,axis=1)
df_test_set = df_test_set.dropna(subset=['Orbit Mean Density (kg/m^3)'],axis=0)

train_dataset = cb.Pool(df_training_set.drop('Orbit Mean Density (kg/m^3)',axis=1), df_training_set['Orbit Mean Density (kg/m^3)']* 10**12) 
valid_dataset = cb.Pool(df_valid_set.drop('Orbit Mean Density (kg/m^3)',axis=1), df_valid_set['Orbit Mean Density (kg/m^3)']* 10**12) 
test_dataset = cb.Pool(df_test_set.drop('Orbit Mean Density (kg/m^3)',axis=1), df_test_set['Orbit Mean Density (kg/m^3)']* 10**12) 

In [ ]:
print(df_training_set.shape)
print(df_valid_set.shape)
print(df_test_set.shape)

#### R2 of predicting only mean

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
test = pd.merge(df_training_set,df_training_set.groupby('FileId')['Orbit Mean Density (kg/m^3)'].mean().rename('Predict_Mean').reset_index(),on='FileId')[['Orbit Mean Density (kg/m^3)','Predict_Mean']]

In [ ]:
r2_score(test.iloc[:,0], test.iloc[:,1])

In [ ]:
#df_training_set = pd.concat( dict((k,res[k]) for k in train_index)).reset_index().rename({'level_0':'FileId'},axis=1)

#### All 

In [ ]:
df_training_set = pd.concat(res.values,ignore_index=True)
df_training_set = df_training_set.drop(['Timestamp_initial'],axis=1)
df_training_set = df_training_set.dropna(subset=['Orbit Mean Density (kg/m^3)'],axis=0)

train_dataset = cb.Pool(df_training_set.drop('Orbit Mean Density (kg/m^3)',axis=1), df_training_set['Orbit Mean Density (kg/m^3)']* 10**12) 

#### Train model

In [ ]:
model = cb.CatBoostRegressor(iterations=100)
grid = {'subsample': [ 0.8, 0.9],
        'depth': [2,4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9],
       }
grid_search_result  = model.grid_search(grid, X = train_dataset)

In [ ]:
model = cb.CatBoostRegressor(loss_function='RMSE',iterations=260,depth=2,eta=0.2,subsample=0.9,l2_leaf_reg=1)
model.fit(train_dataset,eval_set=valid_dataset)

In [ ]:
print(model.score(train_dataset))
print(model.score(valid_dataset))
print(model.score(test_dataset))

In [ ]:
plt.hist(model.predict(train_dataset)/10**12,bins=200)
df_training_set['Orbit Mean Density (kg/m^3)'].hist(bins=200,alpha=0.5)

In [ ]:
pd.DataFrame(train_dataset.get_label()).iloc[:10000:,0].plot(label='Train label')
pd.DataFrame(model.predict(train_dataset)).iloc[:10000,0].plot(label='Model prediction')
plt.legend()

In [ ]:
data = pd.DataFrame({'feature_importance':model.get_feature_importance(), 
                     'feature_names': model.feature_names_}).sort_values(by=['feature_importance'], 
                                                       ascending=False) 
data.head(20)

In [ ]:
model.save_model('data/trained_model/model_by_timestamp_xgboost_second.cbm')

In [ ]:
id = 7630
res[id].reset_index()['Orbit Mean Density (kg/m^3)'].plot()
plt.plot(model.predict(res[id])/10**12,color='red')

In [ ]:
print(model.score(train_dataset))
print(model.score(valid_dataset))
print(model.score(test_dataset))





###  Compare pred

In [ ]:
model = cb.CatBoostRegressor()
model.load_model('data/trained_model/model_by_timestamp_xgboost_test.cbm')
data = pd.DataFrame({'feature_importance':model.get_feature_importance(), 
              'feature_names': model.feature_names_}).sort_values(by=['feature_importance'], 
                                                       ascending=False)
data.head(15)

In [ ]:
model = cb.CatBoostRegressor()
model.load_model('data/trained_model/model_by_timestamp_xgboost_26_02_2025.cbm')
data = pd.DataFrame({'feature_importance':model.get_feature_importance(), 
              'feature_names': model.feature_names_}).sort_values(by=['feature_importance'], 
                                                       ascending=False)
data.head(15)

In [ ]:
model = cb.CatBoostRegressor()
model.load_model('data/trained_model/model_by_timestamp.cbm')
data = pd.DataFrame({'feature_importance':model.get_feature_importance(), 
              'feature_names': model.feature_names_}).sort_values(by=['feature_importance'], 
                                                       ascending=False)
data.head(15)

In [ ]:
# df_pred_by_mean = pd.DataFrame(create_pred_timestamps(df_train['Timestamp'].loc[id]),columns=['Timestamp'])
# df_pred_by_mean['Orbit Mean Density (kg/m^3)'] = model_mean.predict(create_features_and_target_by_dict(id,df_train,omni,sat).drop(['Timestamp', 'mean_Timestamp', 'last_Timestamp','Target']))

In [ ]:
id = 10


In [ ]:
pred_feature = create_features_and_target(id,initial_states,omni,sat,predict_mean=False)
df_pred = pd.DataFrame(model.predict(pred_feature.reset_index().drop(['Timestamp'],axis=1)),
             index=pred_feature.index,columns=['Orbit Mean Density (kg/m^3)']).reset_index()

In [ ]:
df_pred

In [ ]:
df_pred_msis = pd.read_csv('data/data_isaac/sat_density_pred/density_pred'+str(id)+ '.csv',index_col=0)
df_pred_msis['Timestamp'] = pd.to_datetime(df_pred_msis['Timestamp'])

In [ ]:
df_pred_msis.shape

In [ ]:
df_true = sat[id]

In [ ]:
plt.figure()
df_true.set_index('Timestamp')['Orbit Mean Density (kg/m^3)'].plot()
df_pred.set_index('Timestamp')['Orbit Mean Density (kg/m^3)'].plot()
#df_pred_by_mean.set_index('Timestamp')['Orbit Mean Density (kg/m^3)'].plot()
df_pred_msis.set_index('Timestamp')['Density (kg/m3)'].plot()

In [ ]:
df_true

In [ ]:
df_true['Orbit Mean Density (kg/m^3)'].plot()

In [ ]:
df_pred

In [ ]:
df_pred_msis.set_index('Timestamp')['Density (kg/m3)'].plot()


###  Model by MSIS

In [ ]:
from atm_me import PersistenceMSIS, MSISPersistenceAtmosphere, PersistenceModel

In [ ]:
bad_id = []

In [ ]:
model = PersistenceModel(plot_trajectory=False)
for id in df_train.index:
    if os.path.exists('data_isaac/sat_density_pred_no_drag/density_pred'+ str(id) + '.csv'):
        continue
    else:
        bad_id.append(id)
        continue
    omni_data = omni[id].loc[:,['Timestamp', 'f10.7_index', 'ap_index_nT']]
    omni_data['Timestamp'] = pd.to_datetime(omni_data['Timestamp'])
    omni_data = omni_data.ffill()
    try:
        states, densities = model(omni_data,df_train.loc[id].to_dict())
        predictions = model._convert_to_df(states, densities)
        predictions.to_csv('data_isaac/sat_density_pred_no_drag/density_pred'+ str(id) + '.csv')
    except:
        print(id)

In [ ]:
from poliastro.core.elements import coe2rv

In [ ]:
from poliastro.constants.general import GM_earth

In [ ]:
coe2rv(GM_earth,6826.387247,0.003882,87.275306,144.135111,257.314389,102.383270)

In [ ]:
coe2rv(GM_earth,6824.556715,0.004397,87.269194,127.738000,86.817000,273.818611)

In [ ]:
df_train.loc[df_train['Latitude (deg)']<100]

In [ ]:
df_train.loc[bad_id]

In [ ]:
df_train.loc[df_train.index.isin(bad_id)==False]

In [ ]:
len(bad_id)


In [ ]:
from evaluation_me import DensityModelEvaluator

